In [58]:
# pip install pyodbc

In [76]:
import pandas as pd
import pyodbc
import os

In [77]:
# Define the parameters for connect to SQL Server database
driver_name = "ODBC Driver 17 for SQL Server"
server_name = "TIT-20230930FXD\\SQLEXPRESS"
database_name = "brazilian_ecommerce" # using database_name = "master" for create new database
trusted_connection = "yes"

# Define connection string
conn_str = f"DRIVER={driver_name};SERVER={server_name};DATABASE={database_name};Trusted_Connection={trusted_connection}"

In [78]:
# Read CSV file
data_path = r"C:\Users\Admin\Documents\GitHub\data-science-bootcamp9\09_Essential Python for Data Analyst\Project - Automatically Create and add data to a SQL Server\data"
file_name = "olist_geolocation_dataset.csv"
df = pd.read_csv(os.path.join(data_path, file_name), encoding="utf8")
df.info()

geolocation_zip_code_prefix      int64
geolocation_lat                float64
geolocation_lng                float64
geolocation_city                object
geolocation_state               object
dtype: object

In [62]:
df.duplicated().sum()

261831

In [63]:
df = df.drop_duplicates()

In [64]:
df.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [65]:
df = df.astype(object)
df.dtypes

geolocation_zip_code_prefix    object
geolocation_lat                object
geolocation_lng                object
geolocation_city               object
geolocation_state              object
dtype: object

In [66]:
# Convert NaN values to None
df = df.where(pd.notnull(df), None)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 738332 entries, 0 to 1000161
Data columns (total 5 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   geolocation_zip_code_prefix  738332 non-null  object
 1   geolocation_lat              738332 non-null  object
 2   geolocation_lng              738332 non-null  object
 3   geolocation_city             738332 non-null  object
 4   geolocation_state            738332 non-null  object
dtypes: object(5)
memory usage: 33.8+ MB


In [67]:
df.nunique()

geolocation_zip_code_prefix     19015
geolocation_lat                717360
geolocation_lng                717613
geolocation_city                 8011
geolocation_state                  27
dtype: int64

In [68]:
print(df.geolocation_zip_code_prefix.astype(str).str.len().max())
print(df.geolocation_city.astype(str).str.len().max())
print(df.geolocation_state.astype(str).str.len().max())

5
38
2


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 738332 entries, 0 to 1000161
Data columns (total 5 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   geolocation_zip_code_prefix  738332 non-null  object
 1   geolocation_lat              738332 non-null  object
 2   geolocation_lng              738332 non-null  object
 3   geolocation_city             738332 non-null  object
 4   geolocation_state            738332 non-null  object
dtypes: object(5)
memory usage: 33.8+ MB


In [72]:
# drop_table_query = """
#     DROP TABLE IF EXISTS geolocation
# """

# create_table_query = """
#     CREATE TABLE geolocation(    
#         geolocation_zip_code_prefix VARCHAR(50) NOT NULL,
#         geolocation_lat FLOAT NULL,
#         geolocation_lng FLOAT NULL,
#         geolocation_city VARCHAR(50) NULL,
#         geolocation_state VARCHAR(50) NULL
#     );
# """

# with pyodbc.connect(conn_str) as con:
#     con.execute(drop_table_query)
#     con.execute(create_table_query)
#     con.commit()

In [73]:
# Define SQL command for truncate table
truncate_query = """
    TRUNCATE TABLE geolocation
"""

# Execute command SQL for truncate table
with pyodbc.connect(conn_str) as con:
    cursor = con.cursor()
    cursor.execute(truncate_query)
    con.commit()

In [74]:
with pyodbc.connect(conn_str) as con:
    cursor = con.cursor()
    for index, row in df.iterrows():
        insert_query = """
            INSERT INTO geolocation
            VALUES (?, ?, ?, ?, ?)
        """
        # Execute command SQL for insert new data
        cursor.execute(insert_query, tuple(row))
    con.commit()

In [75]:
with pyodbc.connect(conn_str) as con:
    df_query = pd.read_sql("SELECT * FROM geolocation", con)
    
df_query.info()

C:\Users\Admin\AppData\Local\Temp\ipykernel_9936\2495036566.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_query = pd.read_sql("SELECT * FROM geolocation", con)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 738332 entries, 0 to 738331
Data columns (total 5 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   geolocation_zip_code_prefix  738332 non-null  object 
 1   geolocation_lat              738332 non-null  float64
 2   geolocation_lng              738332 non-null  float64
 3   geolocation_city             738332 non-null  object 
 4   geolocation_state            738332 non-null  object 
dtypes: float64(2), object(3)
memory usage: 28.2+ MB
